# Structured Outputs for Consistent LLM Responses

In [68]:
from pydantic_ai import Agent
import nest_asyncio
nest_asyncio.apply()

### Simple structured output

In [69]:
from pydantic import BaseModel

class CityLocation(BaseModel):
    city: str
    country: str

agent = Agent('openai:gpt-4o-mini', result_type=CityLocation)
result = agent.run_sync('Where were the olympics held in 2012?')
print(result.data)


city='London' country='United Kingdom'


### Example of unstructured code generation

In [70]:
from pydantic_ai import Agent

agent = Agent(  
    'openai:gpt-4o-mini',
)

result = agent.run_sync("Write a Python function to add two integers"
                        "together and give an example call")  
print(result.data)

Here's a simple Python function that adds two integers together:

```python
def add_two_integers(a, b):
    """Returns the sum of two integers."""
    return a + b

# Example call
result = add_two_integers(5, 3)
print("The sum is:", result)
```

When you run this code, it will output:

```
The sum is: 8
``` 

You can call the `add_two_integers` function with any two integers to get their sum.


### Structured code generation

In [71]:
class Code_Output(BaseModel):
    code: str
    commentary: str


agent = Agent('openai:gpt-4o-mini', result_type=Code_Output)

result = agent.run_sync("Write a Python function to add two integers" 
                        "together and give an example call")  
#print(result.data)

In [72]:
print(result.data.code)
print()
print(result.data.commentary)

def add_two_integers(a, b):
    """Add two integers together."""
    return a + b

# Example call
result = add_two_integers(3, 5)
print(result)  # Output: 8

The function `add_two_integers` takes two integers as input and returns their sum. In the example call, we add 3 and 5, and the printed result is 8.


### We can run the code easily 

In [73]:
import ipywidgets as widgets

text = widgets.HTML("<h3>Running the code is potentially dangerous</h3>") 
button = widgets.Button(description="Run it, anyway",
                         button_style='danger')
output = widgets.Output(layout={'border': '1px solid black'})
with output:
    print("Code result:")

display(widgets.VBox([text,button, output]))

def run(b):
    with output:
        exec(result.data.code)

button.on_click(run)



### Structured customer feedback

The unstructured version

In [74]:
fb_raw = "shoes.md"
with open(fb_raw, 'r') as f:
    fb = f.read()

print(fb)

# Customer feedback

### CloudStrider Sneakers

1. Stars: 2  
   "The shoes were super comfortable initially, but after just two months, the sole started to peel off! Really disappointed with the durability."  
2. Stars: 1  
   "I ordered my usual size, but these sneakers were way too tight. Had to return them. The sizing is totally off."  
3. Stars: 3  
   "Great for short runs, but they hold onto odors even after washing. Not ideal for heavy use."  
4. Stars: 5  
   "Extremely lightweight and breathable. Perfect for casual jogging. No issues so far!"

### PeakTrek Hikers

1. Stars: 1  
   "These boots claim to be waterproof, but my feet were soaked after just walking through damp grass. Not worth it."  
2. Stars: 2  
   "I slipped multiple times on rocky trails because the soles donâ€™t grip well on wet surfaces. Not safe for serious hikes."  
3. Stars: 4  
   "The boots are sturdy and provide decent ankle support, but I wish the padding was thicker for longer treks."  
4. Stars: 5  

The structured version

In [ ]:
description=f""" 
            Analyse '{fb}' the data provided - it is in Markdown format. 
            The data is about the range of shoes in an online shop each with 
            set of messages from customers giving feedback about the shoes that they have purchased.
            """
            

from pydantic import BaseModel
from enum import Enum

class Sentiment(str, Enum):
    negative = 'negative'
    neutral = 'neutral'
    positive = 'positive'

class feedback_report_shoe(BaseModel):
    product: str
    overall_rating: int  
    issue: str 
    review: str
    sentiment: Sentiment

class feedback_report(BaseModel):
    feedback: list[feedback_report_shoe]

agent = Agent('openai:gpt-4o-mini', result_type=feedback_report)
result = agent.run_sync(description)

In [92]:
print(result.data.model_dump_json(indent=2))

{
  "feedback": [
    {
      "product": "CloudStrider Sneakers",
      "overall_rating": 2,
      "issue": "Durability",
      "review": "The shoes were super comfortable initially, but after just two months, the sole started to peel off! Really disappointed with the durability.",
      "sentiment": "negative"
    },
    {
      "product": "CloudStrider Sneakers",
      "overall_rating": 1,
      "issue": "Sizing",
      "review": "I ordered my usual size, but these sneakers were way too tight. Had to return them. The sizing is totally off.",
      "sentiment": "negative"
    },
    {
      "product": "CloudStrider Sneakers",
      "overall_rating": 3,
      "issue": "Odor retention",
      "review": "Great for short runs, but they hold onto odors even after washing. Not ideal for heavy use.",
      "sentiment": "neutral"
    },
    {
      "product": "CloudStrider Sneakers",
      "overall_rating": 5,
      "issue": "Comfort",
      "review": "Extremely lightweight and breathable. Pe

### Easily converted into a Dataframe

In [77]:
import pandas as pd
df = pd.DataFrame([dict(f) for f in result.data.feedback])
df['sentiment'] = df['sentiment'].apply(lambda x: x.value)
df

,product,overall_rating,issue,review,sentiment
0,CloudStrider Sneakers,2,Durability of sole,"The shoes were super comfortable initially, bu...",negative
1,CloudStrider Sneakers,1,Sizing issue,"I ordered my usual size, but these sneakers we...",negative
2,CloudStrider Sneakers,3,Odor retention,"Great for short runs, but they hold onto odors...",neutral
3,CloudStrider Sneakers,5,None,Extremely lightweight and breathable. Perfect ...,positive
4,PeakTrek Hikers,1,Waterproof claim not met,"These boots claim to be waterproof, but my fee...",negative
5,PeakTrek Hikers,2,Poor grip on wet surfaces,I slipped multiple times on rocky trails becau...,negative
6,PeakTrek Hikers,4,Padding could be thicker,The boots are sturdy and provide decent ankle ...,neutral
7,PeakTrek Hikers,5,None,Used these for a weekend hike in the mountains...,positive
8,SwiftShine Heels,2,Durability issue,"I wore these to a wedding, and the heel broke ...",negative
9,SwiftShine Heels,3,Easily scratched,"While they look stunning, the glossy finish sc...",neutral


### And now we can easily rank the products

In [78]:
average_ratings = df.groupby('product')['overall_rating'].mean()
average_ratings = average_ratings.sort_values(ascending=False)
print(average_ratings)

product
BounceMax Runners        4.75
EcoStep Slip-Ons         4.50
LuxeLace Oxfords         4.50
PeakTrek Hikers          3.00
CloudStrider Sneakers    2.75
SwiftShine Heels         2.50
Name: overall_rating, dtype: float64


In [79]:
from pydantic import BaseModel

class CityLocation(BaseModel):
    city: str
    country: str
    valid: bool = True
    error: str = None

prompt = """If the answer is not a city, 
            return empty strings for the city and country, 
            return valid=False and an error message."""
agent = Agent('openai:gpt-4o-mini', result_type=CityLocation, system_prompt=prompt)


In [80]:
result = agent.run_sync('What is a fish?')
print(result.data)

result = agent.run_sync('In what city was the soccer world cup final last played?')
print(result.data)

city='' country='' valid=False error='The answer provided is not a city.'
city='Doha' country='Qatar' valid=True error=None
